## Obtención de descriptores 
(Javier y Antonio)

Ya que no se aclaran los descriptores que se deben obtener se usa la biblioteca de RDKit y mordred para obtener descriptores 1D y 2D de los compuestos.

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, MolSurf
import warnings
from mordred import Calculator
from mordred import (
    MoriguchiLogP,          # MlogP → clave del artículo (NO en RDKit)
    ZagrebIndex,            # exclusivo de Mordred
    HydrophilicFactor,      # exclusivo
    EccentricConnectivityIndex,
    MolecularDistanceEdge,
    Aromatic,
    Polarizability,
    AcidBase
)

warnings.filterwarnings("ignore")

In [ ]:
# Calculadora ligera: solo descriptores 1D/2D únicos y no redundantes
calc_mordred = Calculator(
    [
        MoriguchiLogP.MlogP,
        ZagrebIndex.ZagrebIndex,
        HydrophilicFactor.HydrophilicFactor,
        EccentricConnectivityIndex.EccentricConnectivityIndex,
        MolecularDistanceEdge.MolecularDistanceEdge,
        Aromatic.AromaticProportion,
        Polarizability.Alpha,
        AcidBase.NumAcid,
        AcidBase.NumBase,
    ],
    ignore_3d=True
)

In [ ]:
def calcular_descriptores_1d_2d(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    descriptores = {}

    # --------------------------------------------------
    # RDKit
    # --------------------------------------------------
    descriptores["PesoMolecular"] = Descriptors.MolWt(mol)
    descriptores["NumAtomos"] = mol.GetNumAtoms()
    descriptores["NumEnlaces"] = mol.GetNumBonds()
    descriptores["NumAtomosPesados"] = Descriptors.HeavyAtomCount(mol)

    atomic_nums = [a.GetAtomicNum() for a in mol.GetAtoms()]
    descriptores["NumAtomosCarbono"] = atomic_nums.count(6)
    descriptores["NumAtomosOxigeno"] = atomic_nums.count(8)
    descriptores["NumAtomosNitrogeno"] = atomic_nums.count(7)
    descriptores["NumAtomosAzufre"] = atomic_nums.count(16)
    descriptores["NumAtomosHidrogeno"] = atomic_nums.count(1)
    descriptores["NumAtomosHalogenos"] = sum(1 for n in atomic_nums if n in [9, 17, 35, 53])
    descriptores["NumHeteroatomos"] = Lipinski.NumHeteroatoms(mol)

    descriptores["HDonadores"] = Lipinski.NumHDonors(mol)
    descriptores["HAceptores"] = Lipinski.NumHAcceptors(mol)
    descriptores["LogP"] = Descriptors.MolLogP(mol)  # RDKit version
    descriptores["TPSA"] = Descriptors.TPSA(mol)
    descriptores["NumRotables"] = Lipinski.NumRotatableBonds(mol)

    descriptores["NumAnillos"] = Descriptors.RingCount(mol)
    descriptores["NumAnillosAromaticos"] = Lipinski.NumAromaticRings(mol)

    carbonos = [a for a in mol.GetAtoms() if a.GetAtomicNum() == 6]
    descriptores["FraccionSP3"] = (
        sum(1 for a in carbonos if a.GetHybridization() == Chem.HybridizationType.SP3)
        / len(carbonos) if carbonos else 0.0
    )

    descriptores["RadioGiro"] = Descriptors.RadiusOfGyration(mol)
    descriptores["AreaSuperficie"] = MolSurf.LabuteASA(mol)
    descriptores["IndiceRefractivo"] = Descriptors.MolMR(mol)

    total = mol.GetNumAtoms()
    if total > 0:
        descriptores["FraccionC"] = descriptores["NumAtomosCarbono"] / total
        descriptores["FraccionO"] = descriptores["NumAtomosOxigeno"] / total
        descriptores["FraccionN"] = descriptores["NumAtomosNitrogeno"] / total
        descriptores["FraccionS"] = descriptores["NumAtomosAzufre"] / total
        descriptores["FraccionH"] = descriptores["NumAtomosHidrogeno"] / total
        descriptores["FraccionHeteroatomos"] = descriptores["NumHeteroatomos"] / total
    else:
        for k in ["FraccionC", "FraccionO", "FraccionN", "FraccionS", "FraccionH", "FraccionHeteroatomos"]:
            descriptores[k] = 0.0

    # --------------------------------------------------
    # Mordred
    # --------------------------------------------------
    try:
        mordred_result = calc_mordred(mol)
        mordred_dict = {}
        for key, val in mordred_result.items():
            if isinstance(val, (int, float)) and not (isinstance(val, float) and np.isnan(val)):
                mordred_dict[str(key)] = float(val)
            else:
                mordred_dict[str(key)] = np.nan
    except Exception:
        # Si falla, llenar con NaN
        mordred_dict = {str(d): np.nan for d in calc_mordred.descriptors}

    # Añadir solo lo que NO tienes en RDKit
    descriptores["MlogP"] = mordred_dict.get("MlogP", np.nan)  # ✅ clave del artículo
    descriptores["ZagrebIndex"] = mordred_dict.get("ZagrebIndex", np.nan)
    descriptores["HydrophilicFactor"] = mordred_dict.get("HydrophilicFactor", np.nan)
    descriptores["EccentricConnectivityIndex"] = mordred_dict.get("EccentricConnectivityIndex", np.nan)
    descriptores["MolecularDistanceEdge"] = mordred_dict.get("MolecularDistanceEdge", np.nan)
    descriptores["AromaticProportion"] = mordred_dict.get("AromaticProportion", np.nan)
    descriptores["Polarizabilidad"] = mordred_dict.get("Alpha", np.nan)  # proxy para DM
    descriptores["NumAcid"] = mordred_dict.get("NumAcid", np.nan)
    descriptores["NumBase"] = mordred_dict.get("NumBase", np.nan)

    return descriptores

In [3]:
# Procesar el dataset
df = pd.read_csv('db/smilesdf.csv')

print(f"Procesando {len(df)} moléculas...")

# Calcular descriptores para cada molécula
descriptores_lista = []
smiles_validos = []
ids_validos = []
ic50_means = []
ic50_stds = []

for idx, row in df.iterrows():
    smiles = row['SMILES']
    descriptores = calcular_descriptores_1d_2d(smiles)
    
    if descriptores is not None:
        descriptores_lista.append(descriptores)
        smiles_validos.append(smiles)
        ids_validos.append(row['id'])
        ic50_means.append(row['IC50_mean'])
        ic50_stds.append(row['IC50_std'])
    else:
        print(f"SMILES inválido: {smiles}")

# Crear DataFrame con los descriptores
if descriptores_lista:
    df_descriptores = pd.DataFrame(descriptores_lista)
    df_descriptores.insert(0, 'SMILES', smiles_validos)
    df_descriptores.insert(0, 'id', ids_validos)
    df_descriptores['IC50_mean'] = ic50_means
    df_descriptores['IC50_std'] = ic50_stds
    
    # Guardar resultados
    archivo_salida = 'db/smile_descriptor.csv'
    df_descriptores.to_csv(archivo_salida, index=False)
else:
    print("❌ No se pudieron calcular descriptores")

Procesando 40 moléculas...


In [4]:
df_descriptores.head()

,id,SMILES,PesoMolecular,NumAtomos,NumEnlaces,NumAtomosCarbono,NumAtomosOxigeno,NumAtomosNitrogeno,NumAtomosAzufre,NumAtomosHidrogeno,...,IndiceRefractivo,NumAtomosPesados,FraccionC,FraccionO,FraccionN,FraccionS,FraccionH,FraccionHeteroatomos,IC50_mean,IC50_std
0,3,c1c(oc(c1)c1sc(nn1)NC)[N+](=O)[O-],226.217,15,16,7,3,4,1,0,...,53.8121,15,0.466667,0.200000,0.266667,0.066667,0.0,0.533333,54.0,0.17
1,4,c1c(oc(c1)c1sc(nn1)NCC)[N+](=O)[O-],240.244,16,17,8,3,4,1,0,...,58.4291,16,0.500000,0.187500,0.250000,0.062500,0.0,0.500000,50.0,0.80
2,5,c1c(oc(c1)c1sc(nn1)N1CC1)[N+](=O)[O-],238.228,16,18,8,3,4,1,0,...,56.4784,16,0.500000,0.187500,0.250000,0.062500,0.0,0.500000,55.5,0.66
3,6,c1c(oc(c1)c1sc(nn1)NCCO)[N+](=O)[O-],256.243,17,18,8,4,4,1,0,...,59.8409,17,0.470588,0.235294,0.235294,0.058824,0.0,0.529412,21.0,0.65
4,7,c1c(oc(c1)c1sc(nn1)NCCCO)[N+](=O)[O-],270.270,18,19,9,4,4,1,0,...,64.4579,18,0.500000,0.222222,0.222222,0.055556,0.0,0.500000,18.0,0.20
